# Purpose of nb:

    Take carbonate chem. system at the 3 April_EXP test stations (where model and observational data are available)
    and compare CO2SYS calculations in MATLAB and MOCSY (fortran wrapper)
    
    
# summary: more or less works, can't seem to get identical const. formulations
## quick 3 station comparison


#### inputs - DIC, ta, temp, sal
    stn nos [42.0, 12.0, 27.0]
    stn_ys = [423,647,505]
    stn_xs = [263, 168, 246]
    dic [ 1871.76062012  1940.02246094  1883.91784668]
    ta [ 1904.46508789  1970.19470215  1913.65466309]
    temp [ 10.7638073   11.32521915  10.66254616]
    sal [ 26.93203926  27.88175011  27.08093452]

#### formulation options for calculation (tried to match as closely as possible)

    [Fortran mocsy:]
    optb = 'u74' *classic formulation of UppstrÃ¶m (1974) for total Boron*
    optk1k2='l' *Lueker et al. (2000) formulations for K1 & K2 (recommended by Dickson et al. 2007)* 
    optKf:  'pf' *Perez & Fraga (1987) formulation for Kf (recommended by Dickson et al., 2007)*
    
    matlab co2sys:
    K1K2 - 10 (Lueker et al 2000), 
    KSO4 - 1 (KSO4 of Dickson 1990a & TB of Uppstrom 1974, PREFERRED)
    
###### So the TB, K1K2 constants are set the same way in both routines, mocsy also sets Kf, Matlab also sets KSO4
    
#### outputs 
    [Fortran mocsy ]
    matlab co2sys = 
    
    pH [ 7.70255729  7.67212223  7.6897756 ]
    pH = 7.7022 7.6717 7.6894

    pco2 =  [ 872.7121784   967.80966505  903.1959605 ]
    pco2 = 832.6472 923.6346 861.6884

    aragonite = [ 0.72107939  0.73056918  0.70450964]
    aragonite =  0.7195 0.7290 0.7030

In [33]:
import sys
sys.path.append('/data/tjarniko/MEOPAR/mocsy')
sys.path.append('/data/tjarniko/MEOPAR/at3/notebooks/carbon_dev/VIZ')
import mocsy
import VIZ_plots as VP

import time
import numpy as np
import matplotlib.pyplot as plt
from oct2py import octave
import seawater as sw
from __future__ import print_function
from numpy import *
from scipy import *
import matplotlib.pyplot as plt

import matplotlib.pyplot as plt
from matplotlib import reload
plt.style.use('seaborn')
import netCDF4 as nc
import numpy as np
import cmocean as cm

""
from salishsea_tools import (
    nc_tools,
    viz_tools,
    geo_tools,
    tidetools
)

%matplotlib inline

In [44]:
w = nc.Dataset('/data/tjarniko/results/may10_a8/SKOG_1d_20160415_20160429_ptrc_T.nc')
wp = nc.Dataset('/data/tjarniko/results/may10_a8/SKOG_1d_20160415_20160429_grid_T.nc')

tDIC = w.variables['dissolved_inorganic_carbon'][0,0,:,:] 
tTA = w.variables['total_alkalinity'][0,0,:,:]
tTEMP = wp.variables['votemper'][0,0,:,:]
tSAL = wp.variables['vosaline'][0,0,:,:]
tZERO = np.zeros_like(tTEMP)



In [60]:
stn_nos = [42.00,12.00,27.00]
stn_ys = [423,647,505]
stn_xs = [263, 168, 246]
surf_DIC = np.zeros_like(stn_nos)
surf_TA = np.zeros_like(stn_nos)
surf_TEMP = np.zeros_like(stn_nos)
surf_sal = np.zeros_like(stn_nos)
surf_pH = np.zeros_like(stn_nos)
surf_ar = np.zeros_like(stn_nos)
surf_pco2 = np.zeros_like(stn_nos)

for i in range(0,len(stn_xs)):
    surf_DIC[i] = tDIC[stn_ys[i],stn_xs[i]] 
    surf_TA[i] = tTA[stn_ys[i],stn_xs[i]] 
    
    print('** STATION **: ', str(stn_nos[i]))
    print('DIC, umol/kg', str(surf_DIC[i]))
    print('TA, umol/kg ', str(surf_TA[i]))
    
    dic = surf_DIC[i] * 1.e-6
#     print('DIC, mol/kg ')
#     print(dic)
    
    alk = surf_TA[i] * 1.e-6
#     print('TA, mol/kg ')
#     print(alk)
    
    print('temp')
    temp = tTEMP[stn_ys[i],stn_xs[i]] 
    surf_TEMP[i] = temp
#     print(temp)
#     print('sal')
    sal = tSAL[stn_ys[i],stn_xs[i]] 
    surf_sal[i] = sal
#    print(sal)
    response_tup = mocsy.mvars(temp=tTEMP[stn_ys[i],stn_xs[i]] , sal=tSAL[stn_ys[i],stn_xs[i]] , alk=alk, dic=dic, sil=0, phos=0, 
                                patm=10, depth=0, lat=50, optcon='mol/kg', optt='Tinsitu', optp='db',
                                optb = 'u74', optk1k2='l', optkf = 'pf', optgas = 'Pinsitu' )
#     response_tup = mocsy.mvars(temp=tTEMP[stn_ys[i],stn_xs[i]] , sal=tSAL[stn_ys[i],stn_xs[i]] , alk=alk, dic=dic, sil=0, phos=0, 
#                                 patm=10, depth=0, lat=50, optcon='mol/kg', optt='Tinsitu', optp='db' )
    pH,pco2,fco2,co2,hco3,co3,OmegaA,OmegaC,BetaD,DENis,p,Tis = response_tup

    print('pH is: ',pH)
    print('pco2 is: ',pco2)
    print('OmegaA is: ',OmegaA)
    
    surf_pH[i] = pH
    surf_ar[i] = OmegaA
    surf_pco2[i] = pco2
#         print('**')  
#         print('pH,pco2,fco2,co2,hco3,co3,OmegaA,OmegaC,BetaD,DENis,p,Tis')
#         print('**')
#         print(pH,pco2,fco2,co2,hco3,co3,OmegaA,OmegaC,BetaD,DENis,p,Tis)
    
    print()



** STATION **:  42.0
DIC, umol/kg 1871.76062012
TA, umol/kg  1904.46508789
temp
pH is:  [ 7.70255729]
pco2 is:  [ 872.7121784]
OmegaA is:  [ 0.72107939]

** STATION **:  12.0
DIC, umol/kg 1940.02246094
TA, umol/kg  1970.19470215
temp
pH is:  [ 7.67212223]
pco2 is:  [ 967.80966505]
OmegaA is:  [ 0.73056918]

** STATION **:  27.0
DIC, umol/kg 1883.91784668
TA, umol/kg  1913.65466309
temp
pH is:  [ 7.6897756]
pco2 is:  [ 903.1959605]
OmegaA is:  [ 0.70450964]



In [62]:
print('mocsy fortran input/output')
print(stn_nos)
print('dic')
print(surf_DIC)
print('ta')
print(surf_TA)
print('temp')
print(surf_TEMP) 
print('sal')
print(surf_sal) 
print('ph')
print(surf_pH)
print('aragonite')
print(surf_ar)
print('pco2')
print(surf_pco2)


mocsy fortran input/output
[42.0, 12.0, 27.0]
dic
[ 1871.76062012  1940.02246094  1883.91784668]
ta
[ 1904.46508789  1970.19470215  1913.65466309]
temp
[ 10.7638073   11.32521915  10.66254616]
sal
[ 26.93203926  27.88175011  27.08093452]
ph
[ 7.70255729  7.67212223  7.6897756 ]
aragonite
[ 0.72107939  0.73056918  0.70450964]
pco2
[ 872.7121784   967.80966505  903.1959605 ]


## Notes on formulation options / Reference

### _MOCSY Fortran_

    Chemistry formulations available in the fortran package:
    
       59   !     ---------
       60   !     optB: choose total boron formulation - UppstrÃ¶m (1974) vs. Lee et al. (2010)
       61   !     ---------
       62   !       -> 'u74' means use classic formulation of UppstrÃ¶m (1974) for total Boron*
       63   !       -> 'l10' means use newer   formulation of Lee et al. (2010) for total Boron
       64   !     ---------
       65   !     optK1K2:
       66   !     ---------
       67   !       -> 'l'   means use Lueker et al. (2000) formulations for K1 & K2 (recommended by Dickson et al. 2007)
       68   !                **** BUT this should only be used when 2 < T < 35 and 19 < S < 43
       69   !       -> 'm10' means use Millero (2010) formulation for K1 & K2 (see Dickson et al., 2007)
       70   !                **** Valid for 0 < T < 50Â°C and 1 < S < 50 psu
       71   !     ----------
       72   !     optKf:
       73   !     ----------
       74   !       -> 'pf' means use Perez & Fraga (1987) formulation for Kf (recommended by Dickson et al., 2007)
       75   !               **** BUT Valid for  9 < T < 33Â°C and 10 < S < 40.
       76   !       -> 'dg' means use Dickson & Riley (1979) formulation for Kf (recommended by Dickson & Goyet, 1994)
       
#### using: optb = 'u74', optk1k2='l', optkf = 'pf'

### _CO2SYS MATLAB_

##### K1K2 dissasociation constants 
        %  (***) Each element must be an integer, 
        %        indicating the K1 K2 dissociation constants that are to be used:
        %   1 = Roy, 1993    T:    0-45  S:  5-45. Total scale. Artificial seawater.
        %   2 = Goyet & Poisson    T:   -1-40  S: 10-50. Seaw. scale. Artificial seawater.
        %   3 = HANSSON refit BY DICKSON AND MILLERO	T:    2-35  S: 20-40. Seaw. scale. Artificial seawater.
        %   4 = MEHRBACH refit BY DICKSON AND MILLERO	T:    2-35  S: 20-40. Seaw. scale. Artificial seawater.
        %   5 = HANSSON and MEHRBACH refit BY DICKSON AND MILLERO	T:    2-35  S: 20-40. Seaw. scale. Artificial seawater.
        %   6 = GEOSECS (i.e., original Mehrbach)					T:    2-35  S: 19-43. NBS scale.   Real seawater.
        %   7 = Peng	(i.e., originam Mehrbach but without XXX)	T:    2-35  S: 19-43. NBS scale.   Real seawater.
        %   8 = Millero, 1979, FOR PURE WATER ONLY (i.e., Sal=0)	T:    0-50  S:     0. 
        %   9 = Cai and Wang, 1998									T:    2-35  S:  0-49. NBS scale.   Real and artificial seawater.
        %  10 = Lueker et al, 2000									T:    2-35  S: 19-43. Total scale. Real seawater.
        %  11 = Mojica Prieto and Millero, 2002.					T:    0-45  S:  5-42. Seaw. scale. Real seawater
        %  12 = Millero et al, 2002						 T: -1.6-35  S: 34-37. Seaw. scale. Field measurements.
        %  13 = Millero et al, 2006						 T:    0-50  S:  1-50. Seaw. scale. Real seawater.
        %  14 = Millero        2010  					 T:    0-50  S:  1-50. Seaw. scale. Real seawater.
        %  15 = Waters, Millero, & Woosley 2014  		 T:    0-50  S:  1-50. Seaw. scale. Real seawater.
        % 
        %  (****) Each element must be an integer that 
        %         indicates the KSO4 dissociation constants that are to be used,
        %         in combination with the formulation of the borate-to-salinity ratio to be used.
        %         Having both these choices in a single argument is somewhat awkward, 
        %         but it maintains syntax compatibility with the previous version.
        %  1 = KSO4 of Dickson 1990a   & TB of Uppstrom 1974  (PREFERRED) 
        %  2 = KSO4 of Khoo et al 1977 & TB of Uppstrom 1974
        %  3 = KSO4 of Dickson 1990a   & TB of Lee 2010
        %  4 = KSO4 of Khoo et al 1977 & TB of Lee 2010

#### using: K1K2 - 10 (Lueker et al), KSO4 - 1 (TB of Uppstrom 1974, PREFERRED)

#### co2sys matlab calculation done on walrus

    reference for script location:
    /Users/terezajana/Sync/phd_other/at3/CO2SYS-MATLAB-master